In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/raw/vineyard_ndvi_2022_2024.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")

df["year"] = df["date"].dt.year
df["doy"] = df["date"].dt.dayofyear
